In [1]:
%%html

<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }
  
  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

### American revolution 

<img src="images/13_colonies.jpg" width="400px" align="right"/>
The American Revolution (1775-83) is also known as the American Revolutionary War and the U.S. War of Independence.

Independance means not beeing under the control of others and beeing able to make one's own decisions. On July 4, 1776,  the Thirteen Colonies declared that they were no longer part of Britain. The British disagreed. A war between Britain and the Thirteen Coloniws began as a result of this disagreement. This war was known as the American Revolution or the War of Independence. (From "Our Land and People", Patricia Shields-Ramsay, Douglas Ramsay)



#### Displaying battles on the map

Step1. Loading python libraries.


In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy import nan as Nan
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py
import time

Step2. Downloading dataset  and examining it.

In [38]:
url = 'https://data.opendatasoft.com/explore/dataset/list-of-american-revolutionary-war-battles@public-us/download/?format=csv&timezone=America/Denver&use_labels_for_header=true'
revolution_data  = pd.read_csv(url,sep=';',error_bad_lines=False)
revolution_data.head()

,Battle,article,Date,State,Outcome,Start Date,Victory,Battle location
0,Battles of Lexington and Concord,https://en.wikipedia.org/wiki/Battles_of_Lexin...,"April 19, 1775",Massachusetts,Patriot victory: British forces raiding Concor...,1775-04-19,Patriot victory,"42.4496388889, -71.2308333333"
1,First Siege of Fort Henry,https://en.wikipedia.org/wiki/Siege_of_Fort_He...,NaN,Virginia,Patriot victory,NaN,Patriot victory,"40.0638888889, -80.725"
2,Battle of Freeman's Farm,https://en.wikipedia.org/wiki/Battles_of_Saratoga,"September 19, 1777",New York,British tactical victory: First of the two Bat...,1777-09-19,NaN,"42.9988888889, -73.6375"
3,Siege of Augusta,https://en.wikipedia.org/wiki/Siege_of_Augusta,NaN,Georgia,Patriot victory,NaN,Patriot victory,"33.4666666667, -81.9666666667"
4,Battle of Videau's Bridge,https://en.wikipedia.org/wiki/Battle_of_Videau...,"January 2, 1782",South Carolina,British victory,1782-01-02,British victory,NaN


Step3. To display battles on the map we neeed longitude and latitude - splitting "Battle location" column into two: "lon" and "lat".
To use a timeline slider we neeed a year  - createing  "Year" column based on first 4 characters of the "Start Date" column.
In adition cleaning some empty values from the dataset(based on  "Battle location" and "Start Date" columns).
Examining dataset again.

In [41]:
revolution_data = revolution_data.dropna(subset=['Battle location'])
revolution_data = revolution_data.dropna(subset=['Start Date'])
revolution_data['lat'] = revolution_data['Battle location'].str.split(', ', expand = True)[0]
revolution_data['lon'] = revolution_data['Battle location'].str.split(', ', expand = True)[1]
revolution_data['Year'] = revolution_data['Start Date'].str[:4]
#fempty row to make legend working
revolution_data.loc[revolution_data.shape[0]]=[' ',' ',' ',' ',' ',' ','Patriot victory',' ','0','0','1783']
revolution_data.head()


,Battle,article,Date,State,Outcome,Start Date,Victory,Battle location,lat,lon,Year
0,Battles of Lexington and Concord,https://en.wikipedia.org/wiki/Battles_of_Lexin...,"April 19, 1775",Massachusetts,Patriot victory: British forces raiding Concor...,1775-04-19,Patriot victory,"42.4496388889, -71.2308333333",42.4496388889,-71.2308333333,1775
2,Battle of Freeman's Farm,https://en.wikipedia.org/wiki/Battles_of_Saratoga,"September 19, 1777",New York,British tactical victory: First of the two Bat...,1777-09-19,Unknown,"42.9988888889, -73.6375",42.9988888889,-73.6375,1777
5,Mount Hope Bay raids,https://en.wikipedia.org/wiki/Mount_Hope_Bay_r...,"May 25-30, 1778",Rhode Island,British victory,1778-05-25,British victory,"41.694, -71.216",41.694,-71.216,1778
6,Capture of Savannah,https://en.wikipedia.org/wiki/Capture_of_Savannah,"December 29, 1778",Georgia,British victory,1778-12-29,British victory,"32.0508333333, -81.1038888889",32.0508333333,-81.1038888889,1778
7,Grand Assault on Gibraltar,https://en.wikipedia.org/wiki/Great_Siege_of_G...,"September 13, 1782",Gibraltar,British victory,1782-09-13,British victory,"36.1397, -5.3551",36.1397,-5.3551,1782


Step4.Displaying battles on the map based on the start date and victory.

In [42]:
init_notebook_mode(connected=True)

years_from_col = set(revolution_data['Year'])
years_ints = sorted(list(years_from_col))
years = [str(year) for year in years_ints]
years.remove('2026')
#print(years)
revolution_data['Victory'] = revolution_data['Victory'].replace(np.nan, 'Unknown', regex=True)
victories = []
for victory in revolution_data['Victory']:
    if victory not in victories: 
        victories.append(victory)       
#print(victories)

figure = {
    'data': [],
    'layout': {},
    'frames': []
}

figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '1774',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}

custom_colors = {
    'Patriot victory': 'rgb(171, 0, 250)',
    'Unknown': 'rgb(150, 99, 0)',
    'British victory': 'rgb(99, 110, 250)',
}

figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}
figure['layout']['title'] = ' American Revolution (1775-83) battles'
figure['layout']['geo']  =  dict(
        scope = 'north america',
        showland = True,
        landcolor = "rgb(212, 212, 212)",
        subunitcolor = "rgb(255, 255, 255)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = True,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation = dict(
                lon = -100
            )
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
)
 

year = 1774
for victory in victories:
    dataset_by_year = revolution_data[revolution_data['Year'] == year]
    dataset_by_year_and_vict = dataset_by_year[dataset_by_year['Victory'] == victory]

    data_dict = {
        'type': 'scattergeo',
        'lon': list(dataset_by_year_and_vict['lon']),
        'lat': list(dataset_by_year_and_vict['lat']),
        'mode': 'markers',
        'text': list(dataset_by_year_and_vict['Battle']),
        'marker': {
            'size': 8, 
            'color': custom_colors[victory]
        },
        'name': victory,
        'legendgroup' : 'group1'
    }
    figure['data'].append(data_dict)
    

for year in years:
    frame = {'data': [], 'name': str(year)}
    for victory in victories:
        dataset_by_year = revolution_data[revolution_data['Year'] == year]
        dataset_by_year_and_vict = dataset_by_year[dataset_by_year['Victory'] == victory]

        data_dict = {
            'type': 'scattergeo',
            'lon': list(dataset_by_year_and_vict['lon']),
            'lat': list(dataset_by_year_and_vict['lat']),
            'mode': 'markers',
            'text': list(dataset_by_year_and_vict['Battle']),
            'marker': {
                'size': 8, 
                'color': custom_colors[victory]
            },
            'name': victory,
            'legendgroup' : 'group'
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

    
figure['layout']['sliders'] = [sliders_dict]
iplot(figure, validate=False)

['Patriot victory', 'Unknown', 'British victory']
